Code Implementation of - **M. M. Islam, G. Karmakar, J. Kamruzzaman, and M. Murshed, “A robust forgery detection method for copy–move and splicing attacks in images,” Electronics, vol. 9, no. 9, 2020. [Online]. Available: https://www.mdpi.com/2079-9292/9/9/1500**

#### Baseline implementation on CASIA 1&2 and Columbia_Uncompressed

In [35]:
import cv2
import numpy as np
import pandas as pd
from scipy.fftpack import dct
from skimage.feature import local_binary_pattern
import os
from skimage.util import view_as_windows
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

In [20]:
def get_patches(image_mat):
    """
    Extract patches rom an image
    :param image_mat: The image as a matrix
    :param stride: The stride of the patch extraction process
    :returns: The patches
    """
    stride=8 #stride is same as window's breadth so that it gives non-overlapping blocks.
    window_shape = (8, 8)
    image_mat=np.array(image_mat)
    
    windows = view_as_windows(image_mat, window_shape, step=stride)
#     print('windows shape:',windows.shape)

    patches = []
    for m in range(windows.shape[0]):
        for n in range(windows.shape[1]):
#             print("window shape: ",windows[m][n].shape)
            patches += [windows[m][n]]
    return windows, patches #sending both the window array and the patches extracted from it.

In [131]:
def feature_sub_channel(sub_channel):
    img_windows, sub_image_blocks = get_patches(sub_channel) #Gives the 8x8 patches/blocks of sub_image.
    
    
#     print("img_windows shape is: ",img_windows.shape)
    img_windows_shape = img_windows.shape
    
    dct_block_windows=[]
    
    for m in range(img_windows.shape[0]):
        for n in range(img_windows.shape[1]):
#             print("Original Block", img_windows[m][n])
            dct_block = dct(img_windows[m][n], type=2, n=None, axis=-1, norm=None, overwrite_x=False)
#             print("DCT Block", dct_block)
            dct_block_windows.append(dct_block)
    
    dct_block_windows = np.asarray(dct_block_windows)
    dct_block_windows = np.reshape(dct_block_windows, newshape=img_windows_shape)
#     print("dct_block_windows shape is: ", dct_block_windows.shape)
#     print("dct_block_windows second block [0][1] is: ", dct_block_windows[0][1])
    
    reconstructed_dct_img = np.zeros(shape=(sub_channel.shape[0], sub_channel.shape[1]))

    i=0
    j=0
    for x in range(dct_block_windows.shape[0]):
        for y in range(dct_block_windows.shape[1]):
            reconstructed_dct_img[i:i + 8, j:j + 8] = dct_block_windows[x,y]
            j=j+8
        i=i+8
        j=0
    
        
    reconstructed_dct_img = abs(reconstructed_dct_img) # Taking absolute of the dct image.
#     print("reconstructed_dct_img shape is: ",reconstructed_dct_img.shape)
#     print("reconstructed_dct_img first block is: ",reconstructed_dct_img[0:8,0:8])
    
    lbp_img = local_binary_pattern(reconstructed_dct_img, 8, 2, method='default') # lbp on whole.
#     print("lbp_img shape is: ",lbp_img.shape)
    
    _, lbp_img_blocks = get_patches(lbp_img) #blocks of lbp image
    
        
    lbp_blocks_array=np.asarray(lbp_img_blocks)
#     print("shape of lbp_blocks_array",lbp_blocks_array.shape)

    _,r,c = lbp_blocks_array.shape #(1536,8,8) for CASIA
#     print("shape of lbp_blocks_array",lbp_blocks_array.shape)
    
    img_mean_list=[]
    for x in range(r):
        for y in range(c):
            pixel_depth_subarr = lbp_blocks_array[:,x,y]
            mean = np.mean(pixel_depth_subarr)
            img_mean_list.append(mean)
    
#     print("Length of img_mean_list: ",len(img_mean_list))
    return img_mean_list


In [132]:
#main function to extract the features.
def feature_extraction(path_to_folder, class_label):
    data_list=[]
    count=0
    for file_name in os.listdir(path_to_folder):
#         if(count>0):
#             break
        path_to_img = os.path.join(path_to_folder,file_name)
        img = cv2.imread(path_to_img)
        
        if np.shape(img) == ():
            continue
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb) #changing to YCrCb color space.
        img_cr = img[:,:,1] # the Cr channel only.
#         print("img_cr shape is: ",img_cr.shape)
        
        Fcr = feature_sub_channel(img_cr)
#         print("fcr shape: ",Fcr.shape)
        
        final_feature = Fcr
        final_feature.insert(0,file_name)
        final_feature.insert(1,class_label)
        data_list.append(final_feature)
        
        count += 1
        
    return(data_list) 

In [168]:
# #CASIA V1.0 dataset
# au_path = "YOUR_PATH/CASIA 1.0 dataset/Au"
# tp_path1 = "YOUR_PATH/CASIA 1.0 dataset/Modified Tp/Tp/CM"
# tp_path2 = "YOUR_PATH/CASIA 1.0 dataset/Modified Tp/Tp/Sp"

# data_list1 = feature_extraction(au_path, 0)
# data_list2 = feature_extraction(tp_path1, 1)
# data_list3 = feature_extraction(tp_path2, 1)

# df = pd.DataFrame(data_list1)
# df = df.append(pd.DataFrame(data_list2), ignore_index=True)
# df = df.append(pd.DataFrame(data_list3), ignore_index=True)
# # #  --------------------------------------------------------------------------------------

# CASIA V2.0 Dataset
# au_path="YOUR_PATH/CASIA2.0_revised/Au"
# tp_path="YOUR_PATH/CASIA2.0_revised/Tp"

# data_list1 = feature_extraction(au_path, 0)
# data_list2 = feature_extraction(tp_path, 1)

# df = pd.DataFrame(data_list1)
# df = df.append(pd.DataFrame(data_list2), ignore_index=True)
# #  --------------------------------------------------------------------------------------

# Columbia dataset
au_path="YOUR_PATH/Columbia_Uncompressed/4cam_auth"
tp_path="YOUR_PATH/Columbia_Uncompressed/4cam_splc"

data_list1 = feature_extraction(au_path, 0)
data_list2 = feature_extraction(tp_path, 1)

df = pd.DataFrame(data_list1)
df = df.append(pd.DataFrame(data_list2), ignore_index=True)
# #  --------------------------------------------------------------------------------------


df.rename(columns = {0: "image_names", 1: "label"}, inplace = True)

scaler_norm = MinMaxScaler() 
df.iloc[:,2:] = scaler_norm.fit_transform(df.iloc[:,2:].to_numpy()) # Normalising the values in dataframe.

In [169]:
df.head()

,image_names,label,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,65
0,canong3_02_sub_05.tif,0,0.060527,0.043104,0.058543,0.171590,0.196873,0.093245,0.090061,0.082571,...,0.123915,0.033466,0.042650,0.058433,0.075300,0.187649,0.242313,0.095483,0.080008,0.077429
1,canonxt_32_sub_05.tif,0,0.332768,0.372694,0.409963,0.295296,0.227577,0.262537,0.327720,0.293200,...,0.360062,0.296317,0.332363,0.386393,0.422207,0.317190,0.263788,0.256202,0.315898,0.309375
2,canonxt_14_sub_04.tif,0,0.423916,0.479795,0.517433,0.405169,0.350693,0.330873,0.350782,0.337591,...,0.387620,0.336337,0.447228,0.477612,0.511860,0.400202,0.352011,0.349267,0.366892,0.342749
3,canong3_02_sub_06.tif,0,0.075004,0.056449,0.094795,0.167862,0.194383,0.074455,0.111890,0.057303,...,0.130620,0.049404,0.054371,0.049513,0.087497,0.198510,0.250536,0.090113,0.063719,0.024183
4,canonxt_14_sub_08.tif,0,0.417658,0.351193,0.456395,0.378651,0.323904,0.304682,0.318419,0.311387,...,0.363272,0.298200,0.417769,0.363026,0.460490,0.406427,0.350761,0.320305,0.312603,0.310821


In [170]:
df.shape

(363, 66)

In [171]:
df.to_csv('Columbia_Uncompressed_baseline_features.csv',index=False)

## SVM CLASSIFICATION CASIA 1.0

In [137]:
array=df.values
x_feature=array[:,2:]
y_label=array[:,1].astype('int')
print(x_feature.shape)
print(y_label.shape)

(1721, 64)
(1721,)


In [157]:
X_train,X_test,Y_train,Y_test=train_test_split(x_feature,y_label,test_size=0.20,random_state=9)

In [152]:
# Random check
model_SVC=SVC(kernel='rbf',C=10,gamma=1)

kfold=KFold(n_splits=10, shuffle=True)
cv_results=cross_val_score(model_SVC,X_train,Y_train,cv=kfold,scoring='accuracy')
msg="%s %f (%f)" % ('Training Accuracy: ',cv_results.mean(),cv_results.std())
print(msg)

Training Accuracy:  0.936835 (0.021885)


In [140]:
# SVM hypertuning using GridSeachCV
model_SVC=SVC()

kfold=KFold(n_splits=10)
param_grid = {'C': [1, 10, 100, 500, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 

grid=GridSearchCV(estimator=model_SVC,param_grid=param_grid,scoring='accuracy',cv=kfold,verbose=3)
grid_result=grid.fit(X_train,Y_train)

print("Best: %f using %s" % (grid_result.best_score_,grid_result.best_params_))

Fitting 10 folds for each of 25 candidates, totalling 250 fits
[CV 1/10] END .........C=1, gamma=1, kernel=rbf;, score=0.891 total time=   0.1s
[CV 2/10] END .........C=1, gamma=1, kernel=rbf;, score=0.906 total time=   0.1s
[CV 3/10] END .........C=1, gamma=1, kernel=rbf;, score=0.877 total time=   0.1s
[CV 4/10] END .........C=1, gamma=1, kernel=rbf;, score=0.884 total time=   0.1s
[CV 5/10] END .........C=1, gamma=1, kernel=rbf;, score=0.855 total time=   0.1s
[CV 6/10] END .........C=1, gamma=1, kernel=rbf;, score=0.899 total time=   0.1s
[CV 7/10] END .........C=1, gamma=1, kernel=rbf;, score=0.905 total time=   0.1s
[CV 8/10] END .........C=1, gamma=1, kernel=rbf;, score=0.920 total time=   0.1s
[CV 9/10] END .........C=1, gamma=1, kernel=rbf;, score=0.883 total time=   0.1s
[CV 10/10] END ........C=1, gamma=1, kernel=rbf;, score=0.876 total time=   0.1s
[CV 1/10] END .......C=1, gamma=0.1, kernel=rbf;, score=0.812 total time=   0.1s
[CV 2/10] END .......C=1, gamma=0.1, kernel=rb

[CV 4/10] END .......C=100, gamma=1, kernel=rbf;, score=0.928 total time=   0.1s
[CV 5/10] END .......C=100, gamma=1, kernel=rbf;, score=0.920 total time=   0.1s
[CV 6/10] END .......C=100, gamma=1, kernel=rbf;, score=0.913 total time=   0.1s
[CV 7/10] END .......C=100, gamma=1, kernel=rbf;, score=0.934 total time=   0.1s
[CV 8/10] END .......C=100, gamma=1, kernel=rbf;, score=0.934 total time=   0.1s
[CV 9/10] END .......C=100, gamma=1, kernel=rbf;, score=0.927 total time=   0.1s
[CV 10/10] END ......C=100, gamma=1, kernel=rbf;, score=0.905 total time=   0.1s
[CV 1/10] END .....C=100, gamma=0.1, kernel=rbf;, score=0.949 total time=   0.1s
[CV 2/10] END .....C=100, gamma=0.1, kernel=rbf;, score=0.920 total time=   0.1s
[CV 3/10] END .....C=100, gamma=0.1, kernel=rbf;, score=0.942 total time=   0.1s
[CV 4/10] END .....C=100, gamma=0.1, kernel=rbf;, score=0.906 total time=   0.1s
[CV 5/10] END .....C=100, gamma=0.1, kernel=rbf;, score=0.899 total time=   0.1s
[CV 6/10] END .....C=100, ga

[CV 6/10] END ......C=1000, gamma=1, kernel=rbf;, score=0.906 total time=   0.1s
[CV 7/10] END ......C=1000, gamma=1, kernel=rbf;, score=0.949 total time=   0.1s
[CV 8/10] END ......C=1000, gamma=1, kernel=rbf;, score=0.927 total time=   0.1s
[CV 9/10] END ......C=1000, gamma=1, kernel=rbf;, score=0.949 total time=   0.1s
[CV 10/10] END .....C=1000, gamma=1, kernel=rbf;, score=0.905 total time=   0.1s
[CV 1/10] END ....C=1000, gamma=0.1, kernel=rbf;, score=0.978 total time=   0.1s
[CV 2/10] END ....C=1000, gamma=0.1, kernel=rbf;, score=0.920 total time=   0.1s
[CV 3/10] END ....C=1000, gamma=0.1, kernel=rbf;, score=0.957 total time=   0.1s
[CV 4/10] END ....C=1000, gamma=0.1, kernel=rbf;, score=0.928 total time=   0.1s
[CV 5/10] END ....C=1000, gamma=0.1, kernel=rbf;, score=0.891 total time=   0.1s
[CV 6/10] END ....C=1000, gamma=0.1, kernel=rbf;, score=0.935 total time=   0.1s
[CV 7/10] END ....C=1000, gamma=0.1, kernel=rbf;, score=0.956 total time=   0.1s
[CV 8/10] END ....C=1000, ga

In [158]:
model_SVC = SVC(C=10,gamma=1, kernel='rbf')
model_SVC.fit(X_train,Y_train) #CASIA1

predictions=model_SVC.predict(X_test)

print(accuracy_score(Y_test,predictions))
print(confusion_matrix(Y_test,predictions))
print(classification_report(Y_test,predictions))

0.9565217391304348
[[150   7]
 [  8 180]]
              precision    recall  f1-score   support

           0       0.95      0.96      0.95       157
           1       0.96      0.96      0.96       188

    accuracy                           0.96       345
   macro avg       0.96      0.96      0.96       345
weighted avg       0.96      0.96      0.96       345



## SVM CLASSIFICATION CASIA 2.0

In [163]:
array=df.values
x_feature=array[:,2:]
y_label=array[:,1].astype('int')
print(x_feature.shape)
print(y_label.shape)

(12614, 64)
(12614,)


In [164]:
X_train,X_test,Y_train,Y_test=train_test_split(x_feature,y_label,test_size=0.20,random_state=9)

In [165]:
# Random check
model_SVC=SVC(kernel='rbf',C=10,gamma=1)

kfold=KFold(n_splits=10, shuffle=True)
cv_results=cross_val_score(model_SVC,X_train,Y_train,cv=kfold,scoring='accuracy')
msg="%s %f (%f)" % ('Training Accuracy: ',cv_results.mean(),cv_results.std())
print(msg)

Training Accuracy:  0.961848 (0.007023)


In [166]:
# SVM hypertuning using GridSeachCV
model_SVC=SVC()

kfold=KFold(n_splits=10)
param_grid = {'C': [1, 10, 100, 500, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 

grid=GridSearchCV(estimator=model_SVC,param_grid=param_grid,scoring='accuracy',cv=kfold,verbose=3)
grid_result=grid.fit(X_train,Y_train)

print("Best: %f using %s" % (grid_result.best_score_,grid_result.best_params_))

Fitting 10 folds for each of 25 candidates, totalling 250 fits
[CV 1/10] END .........C=1, gamma=1, kernel=rbf;, score=0.949 total time=   2.5s
[CV 2/10] END .........C=1, gamma=1, kernel=rbf;, score=0.939 total time=   2.5s
[CV 3/10] END .........C=1, gamma=1, kernel=rbf;, score=0.934 total time=   2.4s
[CV 4/10] END .........C=1, gamma=1, kernel=rbf;, score=0.945 total time=   2.4s
[CV 5/10] END .........C=1, gamma=1, kernel=rbf;, score=0.942 total time=   2.4s
[CV 6/10] END .........C=1, gamma=1, kernel=rbf;, score=0.943 total time=   2.5s
[CV 7/10] END .........C=1, gamma=1, kernel=rbf;, score=0.944 total time=   2.4s
[CV 8/10] END .........C=1, gamma=1, kernel=rbf;, score=0.952 total time=   2.5s
[CV 9/10] END .........C=1, gamma=1, kernel=rbf;, score=0.943 total time=   2.4s
[CV 10/10] END ........C=1, gamma=1, kernel=rbf;, score=0.951 total time=   2.5s
[CV 1/10] END .......C=1, gamma=0.1, kernel=rbf;, score=0.933 total time=   3.5s
[CV 2/10] END .......C=1, gamma=0.1, kernel=rb

[CV 2/10] END .......C=100, gamma=1, kernel=rbf;, score=0.957 total time=   2.0s
[CV 3/10] END .......C=100, gamma=1, kernel=rbf;, score=0.954 total time=   2.0s
[CV 4/10] END .......C=100, gamma=1, kernel=rbf;, score=0.964 total time=   2.0s
[CV 5/10] END .......C=100, gamma=1, kernel=rbf;, score=0.954 total time=   1.9s
[CV 6/10] END .......C=100, gamma=1, kernel=rbf;, score=0.944 total time=   2.0s
[CV 7/10] END .......C=100, gamma=1, kernel=rbf;, score=0.962 total time=   1.9s
[CV 8/10] END .......C=100, gamma=1, kernel=rbf;, score=0.967 total time=   2.0s
[CV 9/10] END .......C=100, gamma=1, kernel=rbf;, score=0.950 total time=   2.1s
[CV 10/10] END ......C=100, gamma=1, kernel=rbf;, score=0.960 total time=   2.0s
[CV 1/10] END .....C=100, gamma=0.1, kernel=rbf;, score=0.961 total time=   1.9s
[CV 2/10] END .....C=100, gamma=0.1, kernel=rbf;, score=0.962 total time=   1.9s
[CV 3/10] END .....C=100, gamma=0.1, kernel=rbf;, score=0.948 total time=   1.9s
[CV 4/10] END .....C=100, ga

[CV 4/10] END ......C=1000, gamma=1, kernel=rbf;, score=0.953 total time=   2.4s
[CV 5/10] END ......C=1000, gamma=1, kernel=rbf;, score=0.949 total time=   2.4s
[CV 6/10] END ......C=1000, gamma=1, kernel=rbf;, score=0.941 total time=   2.5s
[CV 7/10] END ......C=1000, gamma=1, kernel=rbf;, score=0.948 total time=   2.6s
[CV 8/10] END ......C=1000, gamma=1, kernel=rbf;, score=0.960 total time=   2.5s
[CV 9/10] END ......C=1000, gamma=1, kernel=rbf;, score=0.955 total time=   2.5s
[CV 10/10] END .....C=1000, gamma=1, kernel=rbf;, score=0.956 total time=   2.5s
[CV 1/10] END ....C=1000, gamma=0.1, kernel=rbf;, score=0.968 total time=   2.5s
[CV 2/10] END ....C=1000, gamma=0.1, kernel=rbf;, score=0.966 total time=   2.5s
[CV 3/10] END ....C=1000, gamma=0.1, kernel=rbf;, score=0.965 total time=   2.6s
[CV 4/10] END ....C=1000, gamma=0.1, kernel=rbf;, score=0.970 total time=   2.7s
[CV 5/10] END ....C=1000, gamma=0.1, kernel=rbf;, score=0.953 total time=   2.6s
[CV 6/10] END ....C=1000, ga

In [167]:
model_SVC = SVC(C=1000,gamma=0.1, kernel='rbf')
model_SVC.fit(X_train,Y_train) #CASIA2

predictions=model_SVC.predict(X_test)

print(accuracy_score(Y_test,predictions))
print(confusion_matrix(Y_test,predictions))
print(classification_report(Y_test,predictions))

0.9686880697582243
[[1415   39]
 [  40 1029]]
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      1454
           1       0.96      0.96      0.96      1069

    accuracy                           0.97      2523
   macro avg       0.97      0.97      0.97      2523
weighted avg       0.97      0.97      0.97      2523



## SVM CLASSIFICATION Columbia_Uncompressed

In [172]:
array=df.values
x_feature=array[:,2:]
y_label=array[:,1].astype('int')
print(x_feature.shape)
print(y_label.shape)

(363, 64)
(363,)


In [173]:
X_train,X_test,Y_train,Y_test=train_test_split(x_feature,y_label,test_size=0.20,random_state=9)

In [174]:
# Random check
model_SVC=SVC(kernel='rbf',C=10,gamma=1)

kfold=KFold(n_splits=10, shuffle=True)
cv_results=cross_val_score(model_SVC,X_train,Y_train,cv=kfold,scoring='accuracy')
msg="%s %f (%f)" % ('Training Accuracy: ',cv_results.mean(),cv_results.std())
print(msg)

Training Accuracy:  0.772414 (0.078935)


In [175]:
# SVM hypertuning using GridSeachCV
model_SVC=SVC()

kfold=KFold(n_splits=10)
param_grid = {'C': [1, 10, 100, 500, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 

grid=GridSearchCV(estimator=model_SVC,param_grid=param_grid,scoring='accuracy',cv=kfold,verbose=3)
grid_result=grid.fit(X_train,Y_train)

print("Best: %f using %s" % (grid_result.best_score_,grid_result.best_params_))

Fitting 10 folds for each of 25 candidates, totalling 250 fits
[CV 1/10] END .........C=1, gamma=1, kernel=rbf;, score=0.759 total time=   0.0s
[CV 2/10] END .........C=1, gamma=1, kernel=rbf;, score=0.690 total time=   0.0s
[CV 3/10] END .........C=1, gamma=1, kernel=rbf;, score=0.793 total time=   0.0s
[CV 4/10] END .........C=1, gamma=1, kernel=rbf;, score=0.759 total time=   0.0s
[CV 5/10] END .........C=1, gamma=1, kernel=rbf;, score=0.655 total time=   0.0s
[CV 6/10] END .........C=1, gamma=1, kernel=rbf;, score=0.793 total time=   0.0s
[CV 7/10] END .........C=1, gamma=1, kernel=rbf;, score=0.759 total time=   0.0s
[CV 8/10] END .........C=1, gamma=1, kernel=rbf;, score=0.621 total time=   0.0s
[CV 9/10] END .........C=1, gamma=1, kernel=rbf;, score=0.862 total time=   0.0s
[CV 10/10] END ........C=1, gamma=1, kernel=rbf;, score=0.655 total time=   0.0s
[CV 1/10] END .......C=1, gamma=0.1, kernel=rbf;, score=0.828 total time=   0.0s
[CV 2/10] END .......C=1, gamma=0.1, kernel=rb

[CV 3/10] END ....C=100, gamma=0.01, kernel=rbf;, score=0.793 total time=   0.0s
[CV 4/10] END ....C=100, gamma=0.01, kernel=rbf;, score=0.793 total time=   0.0s
[CV 5/10] END ....C=100, gamma=0.01, kernel=rbf;, score=0.690 total time=   0.0s
[CV 6/10] END ....C=100, gamma=0.01, kernel=rbf;, score=0.793 total time=   0.0s
[CV 7/10] END ....C=100, gamma=0.01, kernel=rbf;, score=0.793 total time=   0.0s
[CV 8/10] END ....C=100, gamma=0.01, kernel=rbf;, score=0.621 total time=   0.0s
[CV 9/10] END ....C=100, gamma=0.01, kernel=rbf;, score=0.793 total time=   0.0s
[CV 10/10] END ...C=100, gamma=0.01, kernel=rbf;, score=0.690 total time=   0.0s
[CV 1/10] END ...C=100, gamma=0.001, kernel=rbf;, score=0.759 total time=   0.0s
[CV 2/10] END ...C=100, gamma=0.001, kernel=rbf;, score=0.724 total time=   0.0s
[CV 3/10] END ...C=100, gamma=0.001, kernel=rbf;, score=0.793 total time=   0.0s
[CV 4/10] END ...C=100, gamma=0.001, kernel=rbf;, score=0.759 total time=   0.0s
[CV 5/10] END ...C=100, gamm

[CV 8/10] END ...C=1000, gamma=0.01, kernel=rbf;, score=0.655 total time=   0.0s
[CV 9/10] END ...C=1000, gamma=0.01, kernel=rbf;, score=0.793 total time=   0.0s
[CV 10/10] END ..C=1000, gamma=0.01, kernel=rbf;, score=0.655 total time=   0.0s
[CV 1/10] END ..C=1000, gamma=0.001, kernel=rbf;, score=0.793 total time=   0.0s
[CV 2/10] END ..C=1000, gamma=0.001, kernel=rbf;, score=0.690 total time=   0.0s
[CV 3/10] END ..C=1000, gamma=0.001, kernel=rbf;, score=0.759 total time=   0.0s
[CV 4/10] END ..C=1000, gamma=0.001, kernel=rbf;, score=0.793 total time=   0.0s
[CV 5/10] END ..C=1000, gamma=0.001, kernel=rbf;, score=0.690 total time=   0.0s
[CV 6/10] END ..C=1000, gamma=0.001, kernel=rbf;, score=0.793 total time=   0.0s
[CV 7/10] END ..C=1000, gamma=0.001, kernel=rbf;, score=0.793 total time=   0.0s
[CV 8/10] END ..C=1000, gamma=0.001, kernel=rbf;, score=0.621 total time=   0.0s
[CV 9/10] END ..C=1000, gamma=0.001, kernel=rbf;, score=0.793 total time=   0.0s
[CV 10/10] END .C=1000, gamm

In [176]:
model_SVC = SVC(C=100,gamma=1, kernel='rbf')
model_SVC.fit(X_train,Y_train) 

predictions=model_SVC.predict(X_test)

print(accuracy_score(Y_test,predictions))
print(confusion_matrix(Y_test,predictions))
print(classification_report(Y_test,predictions))

0.726027397260274
[[28 13]
 [ 7 25]]
              precision    recall  f1-score   support

           0       0.80      0.68      0.74        41
           1       0.66      0.78      0.71        32

    accuracy                           0.73        73
   macro avg       0.73      0.73      0.73        73
weighted avg       0.74      0.73      0.73        73

